# Examples of Python k8s api usage

This notebook demonstrates the use of k8s python client package. 

Each k8s operation is show twice.  First, the `kubectl` way of performing the operation is shown.  Next the Python API version of the same operations is shown.

## Install required packages
If needed, uncomment the next cell to install k8s python client package

In [1]:
#!pip install kubernetes

## Imports and setup

In [2]:
from kubernetes import client, config

In [3]:
# Configs can be set in Configuration class directly or using helper utility
config.load_incluster_config()

## List pods in user namespace

### kubectl cli

In [4]:
!kubectl get pod -n kubeflow-user

NAME                                               READY   STATUS    RESTARTS   AGE
ml-pipeline-visualizationserver-6b44c6759f-k5b5w   2/2     Running   2          5h55m
ml-pipeline-ui-artifact-5dd95d555b-f4dwm           2/2     Running   2          5h55m
test-notebook-0                                    2/2     Running   0          102m


### Python API

In [5]:
# get v1 api client
v1 = client.CoreV1Api()

# get pod information
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod('kubeflow-user')
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.42.0.40	kubeflow-user	ml-pipeline-visualizationserver-6b44c6759f-k5b5w
10.42.1.58	kubeflow-user	ml-pipeline-ui-artifact-5dd95d555b-f4dwm
10.42.1.65	kubeflow-user	test-notebook-0


## List deployments in user namespace

### kubectl cli

In [6]:
!kubectl get deploy -n kubeflow-user

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
ml-pipeline-visualizationserver   1/1     1            1           5h56m
ml-pipeline-ui-artifact           1/1     1            1           5h56m


### Python API

In [7]:
appv1 = client.AppsV1Api()
print("listing deployments")
ret = appv1.list_namespaced_deployment('kubeflow-user')
for i in ret.items:
    print(
        f'{i.metadata.name}, replicas: {i.status.replicas}, '
        f'ready replicas: {i.status.ready_replicas}'
    )

listing deployments
ml-pipeline-visualizationserver, replicas: 1, ready replicas: 1
ml-pipeline-ui-artifact, replicas: 1, ready replicas: 1


## Start a pod and then shut it down

### kubectl cli

In [8]:
%%bash
cat <<EOF | kubectl apply -f - -n kubeflow-user
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  containers:
  - name: nginx
    image: nginx:1.14.2
    ports:
    - containerPort: 80
EOF

pod/nginx created


In [9]:
!kubectl get pod -n kubeflow-user

NAME                                               READY   STATUS    RESTARTS   AGE
ml-pipeline-visualizationserver-6b44c6759f-k5b5w   2/2     Running   2          5h56m
ml-pipeline-ui-artifact-5dd95d555b-f4dwm           2/2     Running   2          5h56m
test-notebook-0                                    2/2     Running   0          102m
nginx                                              2/2     Running   0          7s


In [10]:
!kubectl delete pod nginx -n kubeflow-user

pod "nginx" deleted


### Python API

In [11]:
# setup the container definition
nginx_container=client.V1Container(name='nginx')
nginx_container.image = 'nginx:1.14.2'
nginx_container.ports = [client.V1ContainerPort(80)]

# setup pod spec definition
pod_spec = client.V1PodSpec(containers=[nginx_container])

# setup pod metadata
pod_metadata = client.V1ObjectMeta(name='nginx')

# setup Pod definition
pod = client.V1Pod(
    metadata=pod_metadata,
    spec=pod_spec
)

# create the pod
pod_ret = v1.create_namespaced_pod(namespace='kubeflow-user', body=pod)

In [12]:
# get running pods
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod('kubeflow-user')
for i in ret.items:
    print("%s\t%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.status.phase, i.metadata.name))

Listing pods with their IPs:
10.42.0.40	kubeflow-user	Running	ml-pipeline-visualizationserver-6b44c6759f-k5b5w
10.42.1.58	kubeflow-user	Running	ml-pipeline-ui-artifact-5dd95d555b-f4dwm
10.42.1.65	kubeflow-user	Running	test-notebook-0
10.42.0.75	kubeflow-user	Running	nginx


In [13]:
# get information about the nginx pod
print('Containers running in the nginx pod')
for c in pod_ret.spec.containers:
    print(f'  container name: {c.name}')

Containers running in the nginx pod
  container name: nginx
  container name: istio-proxy


In [14]:
# remove the pod
delete_results = v1.delete_namespaced_pod(name='nginx', namespace='kubeflow-user', body=client.V1DeleteOptions())

In [16]:
# get running pods
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod('kubeflow-user')
for i in ret.items:
    print("%s\t%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.status.phase, i.metadata.name))

Listing pods with their IPs:
10.42.0.40	kubeflow-user	Running	ml-pipeline-visualizationserver-6b44c6759f-k5b5w
10.42.1.58	kubeflow-user	Running	ml-pipeline-ui-artifact-5dd95d555b-f4dwm
10.42.1.65	kubeflow-user	Running	test-notebook-0
